<a href="https://colab.research.google.com/github/AliciaAPerez/Death_Analytics_using_CDC_Data_with_Machine_Learning/blob/alicia/death_code_temp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#This code was me testing everything, finalzied version are in the Death_stats files
#there's a lot of extra code and testing code here 

In [2]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.1'
spark_version = 'spark-3.1.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backpor

In [ ]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2021-04-17 09:01:21--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.1’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.33MB/s    in 0.2s    

2021-04-17 09:01:22 (5.33 MB/s) - ‘postgresql-42.2.9.jar.1’ saved [914037/914037]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:
from pyspark import SparkFiles
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType

In [ ]:
url ="https://aperezbucket.s3.us-east-2.amazonaws.com/2015_data.csv"
spark.sparkContext.addFile(url)
death_2015 = spark.read.csv(SparkFiles.get("2015_data.csv"), sep=",", header=True)

# Show DataFrame
death_2015.show()

+---------------+-----------------------+-----------------------+------------------------+--------------+---+---------------+----------+---------------------+-------------+-------------+-------------+--------------------+-----------------------------------+--------------+--------------------+-----------------+--------------+---------------+---------------------+-------+-------------+------------------------------------------------------+----------------------+----------------+----------------+-----------------------+---------------+--------------------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------------------------+-----

In [ ]:
#removing unneded columns
death_2015_clean = death_2015.drop("education_1989_revision","education_reporting_flag","age_substitution_flag","age_recode_52","age_recode_27","age_recode_12", "infant_age_recode_22",
"place_of_death_and_decedents_status","method_of_disposition","autopsy", "activity_code", "place_of_injury_for_causes_w00_y34_except_y06_and_y07_", "358_cause_recode", "113_cause_recode",
"130_infant_cause_recode", "39_cause_recode", "bridged_race_flag", "race_imputation_flag", "race_recode_3", "race_recode_5", "hispanic_origin")

In [ ]:
death_2015_clean.show()

+---------------+-----------------------+--------------+---+---------------+----------+--------------+--------------------+-----------------+--------------+---------------+----------------------+--------------------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-----------------

In [ ]:
#removing any individual under the age of one due to how their recorded in the data per the documentation
#for any subject where "detail_age_type" = 1, the detail_age is the age in years, but "detail_age_type" = 2 through 9, age is months(2), days(4), hours(5), minutes(6), or not stated (9)

death_2015_clean_adults = death_2015_clean.filter((death_2015_clean.detail_age_type == 1))
death_2015_clean_adults.show()

+---------------+-----------------------+--------------+---+---------------+----------+--------------+--------------------+-----------------+--------------+---------------+----------------------+--------------------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-----------------

In [ ]:
#removed anyone under the age of 1 (not in category 1), now removing the "detail_age_type" category as it will no longer be needed
death_2015_adults = death_2015_clean_adults.drop("detail_age_type")
death_2015_adults.show()

+---------------+-----------------------+--------------+---+----------+--------------+--------------------+-----------------+--------------+---------------+----------------------+--------------------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------

In [ ]:
death_2015_adults.printSchema()

root
 |-- resident_status: string (nullable = true)
 |-- education_2003_revision: string (nullable = true)
 |-- month_of_death: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- detail_age: string (nullable = true)
 |-- marital_status: string (nullable = true)
 |-- day_of_week_of_death: string (nullable = true)
 |-- current_data_year: string (nullable = true)
 |-- injury_at_work: string (nullable = true)
 |-- manner_of_death: string (nullable = true)
 |-- icd_code_10th_revision: string (nullable = true)
 |-- number_of_entity_axis_conditions: string (nullable = true)
 |-- entity_condition_1: string (nullable = true)
 |-- entity_condition_2: string (nullable = true)
 |-- entity_condition_3: string (nullable = true)
 |-- entity_condition_4: string (nullable = true)
 |-- entity_condition_5: string (nullable = true)
 |-- entity_condition_6: string (nullable = true)
 |-- entity_condition_7: string (nullable = true)
 |-- entity_condition_8: string (nullable = true)
 |-- entity_

In [ ]:
#row count
death_2015_adults.count()

2694400

In [ ]:
#need to get some data on the figures for the condition columns but they are strings
death_2015_update = death_2015_adults.withColumn("number_of_entity_axis_conditions",col("number_of_entity_axis_conditions").cast("integer")).withColumn("number_of_record_axis_conditions",col("number_of_record_axis_conditions").cast("integer"))
death_2015_update.printSchema()

root
 |-- resident_status: string (nullable = true)
 |-- education_2003_revision: string (nullable = true)
 |-- month_of_death: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- detail_age: string (nullable = true)
 |-- marital_status: string (nullable = true)
 |-- day_of_week_of_death: string (nullable = true)
 |-- current_data_year: string (nullable = true)
 |-- injury_at_work: string (nullable = true)
 |-- manner_of_death: string (nullable = true)
 |-- icd_code_10th_revision: string (nullable = true)
 |-- number_of_entity_axis_conditions: integer (nullable = true)
 |-- entity_condition_1: string (nullable = true)
 |-- entity_condition_2: string (nullable = true)
 |-- entity_condition_3: string (nullable = true)
 |-- entity_condition_4: string (nullable = true)
 |-- entity_condition_5: string (nullable = true)
 |-- entity_condition_6: string (nullable = true)
 |-- entity_condition_7: string (nullable = true)
 |-- entity_condition_8: string (nullable = true)
 |-- entity

In [ ]:
#need to know on average how many underlying conditions there are 20 options are available
death_2015_update.select("number_of_entity_axis_conditions", "number_of_record_axis_conditions").describe().show()

+-------+--------------------------------+--------------------------------+
|summary|number_of_entity_axis_conditions|number_of_record_axis_conditions|
+-------+--------------------------------+--------------------------------+
|  count|                         2694400|                         2694400|
|   mean|               3.103408551068884|               2.953636059976247|
| stddev|              1.9167837941567076|              1.7928775740469587|
|    min|                               1|                               1|
|    max|                              15|                              15|
+-------+--------------------------------+--------------------------------+



In [ ]:
#per documentation of the data, record_axis will be removed as it is for the death certificate, and the entity is pre-known conditions
#for the entity conditions, droppping all but the average of 3, and removing the number of entity as it is no longer needed 
death_2015_cleaned = death_2015_update.drop("number_of_entity_axis_conditions","entity_condition_4","entity_condition_5","entity_condition_6","entity_condition_7","entity_condition_8",
                                            "entity_condition_9","entity_condition_10","entity_condition_11","entity_condition_12","entity_condition_13","entity_condition_14",
                                            "entity_condition_15","entity_condition_16","entity_condition_17","entity_condition_18","entity_condition_19","entity_condition_20",
                                            "number_of_record_axis_conditions", "record_condition_1", "record_condition_2","record_condition_3","record_condition_4","record_condition_5",
                                            "record_condition_6","record_condition_7","record_condition_8","record_condition_9","record_condition_10","record_condition_11","record_condition_12",
                                            "record_condition_13","record_condition_14","record_condition_15","record_condition_16","record_condition_17","record_condition_18","record_condition_19",
                                            "record_condition_20",)

In [ ]:
death_2015_cleaned.show()

+---------------+-----------------------+--------------+---+----------+--------------+--------------------+-----------------+--------------+---------------+----------------------+------------------+------------------+------------------+----+--------------------------+
|resident_status|education_2003_revision|month_of_death|sex|detail_age|marital_status|day_of_week_of_death|current_data_year|injury_at_work|manner_of_death|icd_code_10th_revision|entity_condition_1|entity_condition_2|entity_condition_3|race|hispanic_originrace_recode|
+---------------+-----------------------+--------------+---+----------+--------------+--------------------+-----------------+--------------+---------------+----------------------+------------------+------------------+------------------+----+--------------------------+
|              1|                      3|            01|  M|       084|             M|                   1|             2015|             U|              7|                  I500|            11

In [ ]:
#list of columns
death_2015_cleaned.schema.names

['resident_status',
 'education_2003_revision',
 'month_of_death',
 'sex',
 'detail_age',
 'marital_status',
 'day_of_week_of_death',
 'current_data_year',
 'injury_at_work',
 'manner_of_death',
 'icd_code_10th_revision',
 'entity_condition_1',
 'entity_condition_2',
 'entity_condition_3',
 'race',
 'hispanic_originrace_recode']

In [ ]:
#cleaning up names
death_2015_columns = death_2015_cleaned.withColumnRenamed("resident_status","resident").withColumnRenamed("education_2003_revision","education").withColumnRenamed("detail_age", "age").withColumnRenamed("current_date_year", "year").withColumnRenamed("icd_code_10th_revision", "icd_death_code").withColumnRenamed("hispanic_originrace_recode", "hispanic")
death_2015_columns.show()

+--------+---------+--------------+---+---+--------------+--------------------+-----------------+--------------+---------------+--------------+------------------+------------------+------------------+----+--------+
|resident|education|month_of_death|sex|age|marital_status|day_of_week_of_death|current_data_year|injury_at_work|manner_of_death|icd_death_code|entity_condition_1|entity_condition_2|entity_condition_3|race|hispanic|
+--------+---------+--------------+---+---+--------------+--------------------+-----------------+--------------+---------------+--------------+------------------+------------------+------------------+----+--------+
|       1|        3|            01|  M|084|             M|                   1|             2015|             U|              7|          I500|            11I500|            61L031|              null|  01|       6|
|       1|        6|            01|  M|070|             M|                   2|             2015|             U|              7|          C3

In [ ]:
#what are the types
death_2015_columns.printSchema()

root
 |-- resident: string (nullable = true)
 |-- education: string (nullable = true)
 |-- month_of_death: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- age: string (nullable = true)
 |-- marital_status: string (nullable = true)
 |-- day_of_week_of_death: string (nullable = true)
 |-- current_data_year: string (nullable = true)
 |-- injury_at_work: string (nullable = true)
 |-- manner_of_death: string (nullable = true)
 |-- icd_death_code: string (nullable = true)
 |-- entity_condition_1: string (nullable = true)
 |-- entity_condition_2: string (nullable = true)
 |-- entity_condition_3: string (nullable = true)
 |-- race: string (nullable = true)
 |-- hispanic: string (nullable = true)



In [ ]:
#making the integer columns integer, the death and entity conditions cannot be integers as they have letters as well as numbers
death_2015_int = death_2015_columns.withColumn("resident",col("resident").cast("integer")).withColumn("education",col("education").cast("integer")).withColumn("month_of_death",col("month_of_death").cast("integer")).withColumn("age",col("age").cast("integer")).withColumn("day_of_week_of_death",col("day_of_week_of_death").cast("integer")).withColumn("current_data_year",col("current_data_year").cast("integer")).withColumn("manner_of_death",col("manner_of_death").cast("integer")).withColumn("race",col("race").cast("integer")).withColumn("hispanic",col("hispanic").cast("integer"))
death_2015_int.show()

+--------+---------+--------------+---+---+--------------+--------------------+-----------------+--------------+---------------+--------------+------------------+------------------+------------------+----+--------+
|resident|education|month_of_death|sex|age|marital_status|day_of_week_of_death|current_data_year|injury_at_work|manner_of_death|icd_death_code|entity_condition_1|entity_condition_2|entity_condition_3|race|hispanic|
+--------+---------+--------------+---+---+--------------+--------------------+-----------------+--------------+---------------+--------------+------------------+------------------+------------------+----+--------+
|       1|        3|             1|  M| 84|             M|                   1|             2015|             U|              7|          I500|            11I500|            61L031|              null|   1|       6|
|       1|        6|             1|  M| 70|             M|                   2|             2015|             U|              7|          C3

In [ ]:
#filling in the null values with zero
#we want to maintain the rows, but they will not have other conditions or less than 3
death_2015_final = death_2015_int.fillna(value="0",subset=["entity_condition_1"]).fillna(value="0",subset=["entity_condition_2"]).fillna(value="0",subset=["entity_condition_3"])
death_2015_final.show()

+--------+---------+--------------+---+---+--------------+--------------------+-----------------+--------------+---------------+--------------+------------------+------------------+------------------+----+--------+
|resident|education|month_of_death|sex|age|marital_status|day_of_week_of_death|current_data_year|injury_at_work|manner_of_death|icd_death_code|entity_condition_1|entity_condition_2|entity_condition_3|race|hispanic|
+--------+---------+--------------+---+---+--------------+--------------------+-----------------+--------------+---------------+--------------+------------------+------------------+------------------+----+--------+
|       1|        3|             1|  M| 84|             M|                   1|             2015|             U|              7|          I500|            11I500|            61L031|                 0|   1|       6|
|       1|        6|             1|  M| 70|             M|                   2|             2015|             U|              7|          C3

In [ ]:
#function for other data sets to be cleaned up - only can be used for 2014 and 2015 and newer, older pages use a different education marker
def data_clean_up(data):
  data = data.drop("education_1989_revision","education_reporting_flag","age_substitution_flag","age_recode_52","age_recode_27","age_recode_12", "infant_age_recode_22",
"place_of_death_and_decedents_status","method_of_disposition","autopsy", "activity_code", "place_of_injury_for_causes_w00_y34_except_y06_and_y07_", "358_cause_recode", "113_cause_recode",
"130_infant_cause_recode", "39_cause_recode", "bridged_race_flag", "race_imputation_flag", "race_recode_3", "race_recode_5", "hispanic_origin");
  data = data.filter((data.detail_age_type == 1))
  data = data.drop("detail_age_type")
  data = data.drop("number_of_entity_axis_conditions","entity_condition_4","entity_condition_5","entity_condition_6","entity_condition_7","entity_condition_8",
                                            "entity_condition_9","entity_condition_10","entity_condition_11","entity_condition_12","entity_condition_13","entity_condition_14",
                                            "entity_condition_15","entity_condition_16","entity_condition_17","entity_condition_18","entity_condition_19","entity_condition_20",
                                            "number_of_record_axis_conditions", "record_condition_1", "record_condition_2","record_condition_3","record_condition_4","record_condition_5",
                                            "record_condition_6","record_condition_7","record_condition_8","record_condition_9","record_condition_10","record_condition_11","record_condition_12",
                                            "record_condition_13","record_condition_14","record_condition_15","record_condition_16","record_condition_17","record_condition_18","record_condition_19",
                                            "record_condition_20",)
  data = data.withColumnRenamed("resident_status","resident").withColumnRenamed("education_2003_revision","education").withColumnRenamed("detail_age", "age").withColumnRenamed("current_date_year", "year").withColumnRenamed("icd_code_10th_revision", "icd_death_code").withColumnRenamed("hispanic_originrace_recode", "hispanic")
  data = data.withColumn("resident",col("resident").cast("integer")).withColumn("education",col("education").cast("integer")).withColumn("month_of_death",col("month_of_death").cast("integer")).withColumn("age",col("age").cast("integer")).withColumn("day_of_week_of_death",col("day_of_week_of_death").cast("integer")).withColumn("current_data_year",col("current_data_year").cast("integer")).withColumn("manner_of_death",col("manner_of_death").cast("integer")).withColumn("race",col("race").cast("integer")).withColumn("hispanic",col("hispanic").cast("integer"))
  data = data.fillna(value="0",subset=["entity_condition_1"]).fillna(value="0",subset=["entity_condition_2"]).fillna(value="0",subset=["entity_condition_3"])
  data = data.na.replace(["Y", "N", "U"], ["1","2","3"], "injury_at_work")
  data = data.na.replace(["M", "F"], ["1","2"], "sex")
  data = data.na.replace(["S", "M", "W", "D", "U"], ["1","2", "3", "4", "5"], "marital_status")
  data = data.select("icd_death_code","resident","education","month_of_death","sex", "age","marital_status","day_of_week_of_death","current_data_year","injury_at_work","manner_of_death","race","hispanic","entity_condition_1","entity_condition_2","entity_condition_3")
  return data

In [ ]:
#function for other data sets to be cleaned up - only can be used for 2014 and 2015 and newer, older pages use a different education marker
def data_clean_up2(data):
  data = data.drop("education_2003_revision","education_reporting_flag","age_substitution_flag","age_recode_52","age_recode_27","age_recode_12", "infant_age_recode_22",
"place_of_death_and_decedents_status","method_of_disposition","autopsy", "activity_code", "place_of_injury_for_causes_w00_y34_except_y06_and_y07_", "358_cause_recode", "113_cause_recode",
"130_infant_cause_recode", "39_cause_recode", "bridged_race_flag", "race_imputation_flag", "race_recode_3", "race_recode_5", "hispanic_origin");
  data = data.filter((data.detail_age_type == 1))
  data = data.drop("detail_age_type")
  data = data.drop("number_of_entity_axis_conditions","entity_condition_4","entity_condition_5","entity_condition_6","entity_condition_7","entity_condition_8",
                                            "entity_condition_9","entity_condition_10","entity_condition_11","entity_condition_12","entity_condition_13","entity_condition_14",
                                            "entity_condition_15","entity_condition_16","entity_condition_17","entity_condition_18","entity_condition_19","entity_condition_20",
                                            "number_of_record_axis_conditions", "record_condition_1", "record_condition_2","record_condition_3","record_condition_4","record_condition_5",
                                            "record_condition_6","record_condition_7","record_condition_8","record_condition_9","record_condition_10","record_condition_11","record_condition_12",
                                            "record_condition_13","record_condition_14","record_condition_15","record_condition_16","record_condition_17","record_condition_18","record_condition_19",
                                            "record_condition_20",)
  data = data.withColumnRenamed("resident_status","resident").withColumnRenamed("education_1989_revision","education").withColumnRenamed("detail_age", "age").withColumnRenamed("current_date_year", "year").withColumnRenamed("icd_code_10th_revision", "icd_death_code").withColumnRenamed("hispanic_originrace_recode", "hispanic")
  data = data.withColumn("resident",col("resident").cast("integer")).withColumn("education",col("education").cast("integer")).withColumn("month_of_death",col("month_of_death").cast("integer")).withColumn("age",col("age").cast("integer")).withColumn("day_of_week_of_death",col("day_of_week_of_death").cast("integer")).withColumn("current_data_year",col("current_data_year").cast("integer")).withColumn("manner_of_death",col("manner_of_death").cast("integer")).withColumn("race",col("race").cast("integer")).withColumn("hispanic",col("hispanic").cast("integer"))
  data = data.fillna(value="0",subset=["entity_condition_1"]).fillna(value="0",subset=["entity_condition_2"]).fillna(value="0",subset=["entity_condition_3"])
  data = data.na.replace([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,99], [1,1,1,1,1,1,1,1,1,2,2,2,3,4,4,4,6,7,9], "education")
  data = data.na.replace(["Y", "N", "U"], ["1","2","3"], "injury_at_work")
  data = data.na.replace(["M", "F"], ["1","2"], "sex")
  data = data.na.replace(["S", "M", "W", "D", "U"], ["1","2", "3", "4", "5"], "marital_status")
  data = data.select("icd_death_code","resident","education","month_of_death","sex", "age","marital_status","day_of_week_of_death","current_data_year","injury_at_work","manner_of_death","race","hispanic","entity_condition_1","entity_condition_2","entity_condition_3")
  return data

In [ ]:
url ="https://aperezbucket.s3.us-east-2.amazonaws.com/2015_data.csv"
spark.sparkContext.addFile(url)
death_2015 = spark.read.csv(SparkFiles.get("2015_data.csv"), sep=",", header=True)

# Show DataFrame
death_2015_final = data_clean_up(death_2015)
death_2015_final.show()

+--------------+--------+---------+--------------+---+---+--------------+--------------------+-----------------+--------------+---------------+----+--------+------------------+------------------+------------------+
|icd_death_code|resident|education|month_of_death|sex|age|marital_status|day_of_week_of_death|current_data_year|injury_at_work|manner_of_death|race|hispanic|entity_condition_1|entity_condition_2|entity_condition_3|
+--------------+--------+---------+--------------+---+---+--------------+--------------------+-----------------+--------------+---------------+----+--------+------------------+------------------+------------------+
|          I500|       1|        3|             1|  1| 84|             2|                   1|             2015|             3|              7|   1|       6|            11I500|            61L031|                 0|
|          C349|       1|        6|             1|  1| 70|             2|                   2|             2015|             3|             

In [ ]:
url ="https://aperezbucket.s3.us-east-2.amazonaws.com/2014_data.csv"
spark.sparkContext.addFile(url)
death_2014 = spark.read.csv(SparkFiles.get("2014_data.csv"), sep=",", header=True)

#run the function for clean_up and show that dataframe
death_2014_final = data_clean_up(death_2014)
death_2014_final.show()

+--------------+--------+---------+--------------+---+---+--------------+--------------------+-----------------+--------------+---------------+----+--------+------------------+------------------+------------------+
|icd_death_code|resident|education|month_of_death|sex|age|marital_status|day_of_week_of_death|current_data_year|injury_at_work|manner_of_death|race|hispanic|entity_condition_1|entity_condition_2|entity_condition_3|
+--------------+--------+---------+--------------+---+---+--------------+--------------------+-----------------+--------------+---------------+----+--------+------------------+------------------+------------------+
|           I64|       1|        2|             1|  1| 87|             2|                   4|             2014|             3|              7|   1|       6|             11I64|                 0|                 0|
|          I250|       1|        2|             1|  1| 58|             4|                   3|             2014|             3|             

In [ ]:
url ="https://aperezbucket.s3.us-east-2.amazonaws.com/2013_data.csv"
spark.sparkContext.addFile(url)
death_2013 = spark.read.csv(SparkFiles.get("2013_data.csv"), sep=",", header=True)

#run the function for clean_up and show that dataframe
death_2013_final = data_clean_up2(death_2013)
death_2013_final.show()

+--------------+--------+---------+--------------+---+---+--------------+--------------------+-----------------+--------------+---------------+----+--------+------------------+------------------+------------------+
|icd_death_code|resident|education|month_of_death|sex|age|marital_status|day_of_week_of_death|current_data_year|injury_at_work|manner_of_death|race|hispanic|entity_condition_1|entity_condition_2|entity_condition_3|
+--------------+--------+---------+--------------+---+---+--------------+--------------------+-----------------+--------------+---------------+----+--------+------------------+------------------+------------------+
|           F03|       1|        4|             1|  1| 90|             2|                   4|             2013|             3|              7|   1|       6|             11F03|             61I10|            62K579|
|           F03|       1|        3|             1|  2| 80|             3|                   3|             2013|             3|             

In [ ]:
url ="https://aperezbucket.s3.us-east-2.amazonaws.com/2012_data.csv"
spark.sparkContext.addFile(url)
death_2012 = spark.read.csv(SparkFiles.get("2012_data.csv"), sep=",", header=True)
death_2012 = death_2012.withColumnRenamed("icd_code_10","icd_death_code")
#run the function for clean_up and show that dataframe
death_2012_final = data_clean_up2(death_2012)
death_2012_final.show()

+--------------+--------+---------+--------------+---+---+--------------+--------------------+-----------------+--------------+---------------+----+--------+------------------+------------------+------------------+
|icd_death_code|resident|education|month_of_death|sex|age|marital_status|day_of_week_of_death|current_data_year|injury_at_work|manner_of_death|race|hispanic|entity_condition_1|entity_condition_2|entity_condition_3|
+--------------+--------+---------+--------------+---+---+--------------+--------------------+-----------------+--------------+---------------+----+--------+------------------+------------------+------------------+
|          C349|       1|        3|             1|  2| 59|             3|                   1|             2012|             3|              7|   1|       6|            11C349|                 0|                 0|
|           F03|       1|        3|             1|  2| 90|             3|                   2|             2012|             3|             

In [ ]:
url ="https://aperezbucket.s3.us-east-2.amazonaws.com/2011_data.csv"
spark.sparkContext.addFile(url)
death_2011 = spark.read.csv(SparkFiles.get("2011_data.csv"), sep=",", header=True)

#run the function for clean_up and show that dataframe
death_2011_final = data_clean_up2(death_2011)
death_2011_final.show()

+--------------+--------+---------+--------------+---+---+--------------+--------------------+-----------------+--------------+---------------+----+--------+------------------+------------------+------------------+
|icd_death_code|resident|education|month_of_death|sex|age|marital_status|day_of_week_of_death|current_data_year|injury_at_work|manner_of_death|race|hispanic|entity_condition_1|entity_condition_2|entity_condition_3|
+--------------+--------+---------+--------------+---+---+--------------+--------------------+-----------------+--------------+---------------+----+--------+------------------+------------------+------------------+
|          C859|       2|        3|             1|  2| 65|             2|                   2|             2011|             3|              7|   1|       6|            11C859|                 0|                 0|
|          C509|       1|        1|             1|  2| 57|             1|                   7|             2011|             3|             

In [ ]:
url ="https://aperezbucket.s3.us-east-2.amazonaws.com/2010_data.csv"
spark.sparkContext.addFile(url)
death_2010 = spark.read.csv(SparkFiles.get("2010_data.csv"), sep=",", header=True)

#run the function for clean_up and show that dataframe
death_2010_final = data_clean_up2(death_2010)
death_2010_final.show()

+--------------+--------+---------+--------------+---+---+--------------+--------------------+-----------------+--------------+---------------+----+--------+------------------+------------------+------------------+
|icd_death_code|resident|education|month_of_death|sex|age|marital_status|day_of_week_of_death|current_data_year|injury_at_work|manner_of_death|race|hispanic|entity_condition_1|entity_condition_2|entity_condition_3|
+--------------+--------+---------+--------------+---+---+--------------+--------------------+-----------------+--------------+---------------+----+--------+------------------+------------------+------------------+
|          G049|       2|        3|             1|  2| 68|             4|                   6|             2010|             3|              7|   3|       8|            11G049|             61F03|                 0|
|           C37|       1|        1|             1|  2| 12|             1|                   1|             2010|             3|             

In [ ]:
url ="https://aperezbucket.s3.us-east-2.amazonaws.com/2009_data.csv"
spark.sparkContext.addFile(url)
death_2009 = spark.read.csv(SparkFiles.get("2009_data.csv"), sep=",", header=True)

#run the function for clean_up and show that dataframe
death_2009_final = data_clean_up2(death_2009)
death_2009_final.show()

+--------------+--------+---------+--------------+---+---+--------------+--------------------+-----------------+--------------+---------------+----+--------+------------------+------------------+------------------+
|icd_death_code|resident|education|month_of_death|sex|age|marital_status|day_of_week_of_death|current_data_year|injury_at_work|manner_of_death|race|hispanic|entity_condition_1|entity_condition_2|entity_condition_3|
+--------------+--------+---------+--------------+---+---+--------------+--------------------+-----------------+--------------+---------------+----+--------+------------------+------------------+------------------+
|          F329|       1|        3|             1|  2| 80|             2|                   2|             2009|             3|              7|  68|       8|             11E46|            21F329|             61I64|
|          C920|       1|        3|             1|  1| 77|             3|                   1|             2009|             3|             

In [ ]:
url ="https://aperezbucket.s3.us-east-2.amazonaws.com/2008_data.csv"
spark.sparkContext.addFile(url)
death_2008 = spark.read.csv(SparkFiles.get("2008_data.csv"), sep=",", header=True)

#run the function for clean_up and show that dataframe
death_2008_final = data_clean_up2(death_2008)
death_2008_final.show()

+--------------+--------+---------+--------------+---+---+--------------+--------------------+-----------------+--------------+---------------+----+--------+------------------+------------------+------------------+
|icd_death_code|resident|education|month_of_death|sex|age|marital_status|day_of_week_of_death|current_data_year|injury_at_work|manner_of_death|race|hispanic|entity_condition_1|entity_condition_2|entity_condition_3|
+--------------+--------+---------+--------------+---+---+--------------+--------------------+-----------------+--------------+---------------+----+--------+------------------+------------------+------------------+
|          C159|       1|        3|             1|  1| 65|             2|                   1|             2008|             3|              7|   1|       6|            11C159|                 0|                 0|
|          I251|       1|        2|             1|  1| 78|             2|                   7|             2008|             3|             

In [ ]:
url ="https://aperezbucket.s3.us-east-2.amazonaws.com/2007_data.csv"
spark.sparkContext.addFile(url)
death_2007 = spark.read.csv(SparkFiles.get("2007_data.csv"), sep=",", header=True)

#run the function for clean_up and show that dataframe
death_2007_final = data_clean_up2(death_2007)
death_2007_final.show()

+--------------+--------+---------+--------------+---+---+--------------+--------------------+-----------------+--------------+---------------+----+--------+------------------+------------------+------------------+
|icd_death_code|resident|education|month_of_death|sex|age|marital_status|day_of_week_of_death|current_data_year|injury_at_work|manner_of_death|race|hispanic|entity_condition_1|entity_condition_2|entity_condition_3|
+--------------+--------+---------+--------------+---+---+--------------+--------------------+-----------------+--------------+---------------+----+--------+------------------+------------------+------------------+
|          C259|       3|        1|             1|  1| 72|             2|                   2|             2007|             3|              7|   1|       6|            11C259|                 0|                 0|
|           X74|       1|        3|             1|  1| 35|             2|                   2|             2007|             3|             

In [ ]:
url ="https://aperezbucket.s3.us-east-2.amazonaws.com/2006_data.csv"
spark.sparkContext.addFile(url)
death_2006 = spark.read.csv(SparkFiles.get("2006_data.csv"), sep=",", header=True)

#run the function for clean_up and show that dataframe
death_2006_final = data_clean_up2(death_2006)
death_2006_final.show()

+--------------+--------+---------+--------------+---+---+--------------+--------------------+-----------------+--------------+---------------+----+--------+------------------+------------------+------------------+
|icd_death_code|resident|education|month_of_death|sex|age|marital_status|day_of_week_of_death|current_data_year|injury_at_work|manner_of_death|race|hispanic|entity_condition_1|entity_condition_2|entity_condition_3|
+--------------+--------+---------+--------------+---+---+--------------+--------------------+-----------------+--------------+---------------+----+--------+------------------+------------------+------------------+
|          I500|       1|        2|             1|  2| 84|             3|                   2|             2006|             3|           null|   1|       6|            11I500|                 0|                 0|
|          I251|       1|        1|             1|  2| 93|             3|                   1|             2006|             3|           nu

In [ ]:
url ="https://aperezbucket.s3.us-east-2.amazonaws.com/2005_data.csv"
spark.sparkContext.addFile(url)
death_2005 = spark.read.csv(SparkFiles.get("2005_data.csv"), sep=",", header=True)

#run the function for clean_up and show that dataframe
death_2005_final = data_clean_up2(death_2005)
death_2005_final.show()

+--------------+--------+---------+--------------+---+---+--------------+--------------------+-----------------+--------------+---------------+----+--------+------------------+------------------+------------------+
|icd_death_code|resident|education|month_of_death|sex|age|marital_status|day_of_week_of_death|current_data_year|injury_at_work|manner_of_death|race|hispanic|entity_condition_1|entity_condition_2|entity_condition_3|
+--------------+--------+---------+--------------+---+---+--------------+--------------------+-----------------+--------------+---------------+----+--------+------------------+------------------+------------------+
|          C439|       1|        2|             1|  2| 45|             2|                   2|             2005|             3|              7|   1|       6|            11C439|                 0|                 0|
|          J439|       1|        4|             1|  1| 61|             4|                   7|             2005|             3|             

In [ ]:
import pandas as pd

In [ ]:
df = death_2015_final.select("*").toPandas()

In [ ]:
df

,resident,education,month_of_death,sex,age,marital_status,day_of_week_of_death,current_data_year,injury_at_work,manner_of_death,icd_death_code,entity_condition_1,entity_condition_2,entity_condition_3,race,hispanic
0,1,3.0,1,1,84,2,1,2015,3,7.0,I500,11I500,61L031,0,1,6
1,1,6.0,1,1,70,2,2,2015,3,7.0,C349,11I469,21R042,31C349,1,6
2,1,3.0,1,2,91,3,6,2015,3,7.0,G309,11G309,0,0,1,6
3,1,3.0,1,2,40,1,5,2015,2,3.0,X91,11T71,12X91,21T71,3,8
4,1,5.0,1,2,89,3,5,2015,3,7.0,I250,11I250,21S720,61X590,1,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2694395,1,9.0,10,2,46,5,7,2015,3,7.0,E059,11I469,21I48,31I500,2,7
2694396,1,9.0,12,2,48,5,6,2015,3,7.0,I251,11R092,21I251,61F179,1,6
2694397,4,9.0,12,1,33,5,1,2015,3,7.0,F102,11F102,0,0,1,5
2694398,4,9.0,7,1,83,5,7,2015,3,7.0,I250,11R263,21I119,22I679,2,7


In [ ]:
df2 = df.value_counts("icd_death_code")
df2

icd_death_code
I251    164036
C349    152667
J449    114173
I219    112288
G309    108409
         ...  
O059         1
O065         1
D303         1
O069         1
A009         1
Length: 3582, dtype: int64

In [ ]:
print(df2.to_string())

icd_death_code
I251    164036
C349    152667
J449    114173
I219    112288
G309    108409
F03     107879
I500     64918
I64      64692
I250     60954
J189     45538
C509     42004
C259     41330
C189     40301
A419     38013
E149     30581
C61      28874
I119     27982
G20      27806
C80      27547
I48      23885
J440     21607
I10      20727
X44      20170
X42      20005
N185     19886
J690     19813
K746     18679
I429     17778
C679     16212
K703     15259
W19      15167
C159     15081
C719     15029
I350     14733
E142     14727
I469     14538
C64      14246
J841     13984
C56      13949
I619     13888
I110     13854
X74      13383
E119     12486
F019     12310
N390     12153
C900     12143
X70      11891
V892     11846
I255     11787
X95      11548
W18      11367
C859     11150
I120     11124
R99      11067
I259     10783
I694     10485
C169     10356
N19      10228
C220     10082
I509     10044
E785      9698
N189      9627
C920      9572
C229      9428
I629      8982
K922      

In [ ]:
df3 = pd.DataFrame(df2)
df3

,0
icd_death_code,
I251,164036
C349,152667
J449,114173
I219,112288
G309,108409
...,...
O059,1
O065,1
D303,1


In [ ]:
#there are 2,694,400 rows of data, and 3582 unique causes of death, if a cause of death affects 10,000 people, that is 0.37% of the listed rows. 
#for machine learning there are too many causes for the data to be properly evaluated to make a guess. 
#picked every cause that was above 10,000, to see how many there are, and will continue to trim if necessary

In [ ]:
# from pyspark.sql.functions import monotonically_increasing_id
df3.insert(0, 'ID', range(1, 1 +len(df3)))
df3

,ID,0
icd_death_code,,
I251,1,164036
C349,2,152667
J449,3,114173
I219,4,112288
G309,5,108409
...,...,...
O059,3578,1
O065,3579,1
D303,3580,1


In [ ]:
df3 = df3.reset_index()

In [ ]:
#i want the first 60
df3.head(62)

,icd_death_code,ID,0
0,I251,1,164036
1,C349,2,152667
2,J449,3,114173
3,I219,4,112288
4,G309,5,108409
...,...,...,...
57,N19,58,10228
58,C220,59,10082
59,I509,60,10044
60,E785,61,9698


In [ ]:
df60 = df3.head(60)
df60 = pd.DataFrame(df60)
df60.head()

,icd_death_code,ID,0
0,I251,1,164036
1,C349,2,152667
2,J449,3,114173
3,I219,4,112288
4,G309,5,108409


In [ ]:
code_list_60 = df60["icd_death_code"].tolist()
print(code_list_60)

['I251', 'C349', 'J449', 'I219', 'G309', 'F03', 'I500', 'I64', 'I250', 'J189', 'C509', 'C259', 'C189', 'A419', 'E149', 'C61', 'I119', 'G20', 'C80', 'I48', 'J440', 'I10', 'X44', 'X42', 'N185', 'J690', 'K746', 'I429', 'C679', 'K703', 'W19', 'C159', 'C719', 'I350', 'E142', 'I469', 'C64', 'J841', 'C56', 'I619', 'I110', 'X74', 'E119', 'F019', 'N390', 'C900', 'X70', 'V892', 'I255', 'X95', 'W18', 'C859', 'I120', 'R99', 'I259', 'I694', 'C169', 'N19', 'C220', 'I509']


In [ ]:
df55 = df3.head(55)
df55 = pd.DataFrame(df55)
code_list_55 = df55["icd_death_code"].tolist()
print(code_list_55)

['I251', 'C349', 'J449', 'I219', 'G309', 'F03', 'I500', 'I64', 'I250', 'J189', 'C509', 'C259', 'C189', 'A419', 'E149', 'C61', 'I119', 'G20', 'C80', 'I48', 'J440', 'I10', 'X44', 'X42', 'N185', 'J690', 'K746', 'I429', 'C679', 'K703', 'W19', 'C159', 'C719', 'I350', 'E142', 'I469', 'C64', 'J841', 'C56', 'I619', 'I110', 'X74', 'E119', 'F019', 'N390', 'C900', 'X70', 'V892', 'I255', 'X95', 'W18', 'C859', 'I120', 'R99', 'I259']


In [ ]:
df50 = df3.head(50)
df50 = pd.DataFrame(df50)
code_list_50 = df50["icd_death_code"].tolist()
print(code_list_50)

['I251', 'C349', 'J449', 'I219', 'G309', 'F03', 'I500', 'I64', 'I250', 'J189', 'C509', 'C259', 'C189', 'A419', 'E149', 'C61', 'I119', 'G20', 'C80', 'I48', 'J440', 'I10', 'X44', 'X42', 'N185', 'J690', 'K746', 'I429', 'C679', 'K703', 'W19', 'C159', 'C719', 'I350', 'E142', 'I469', 'C64', 'J841', 'C56', 'I619', 'I110', 'X74', 'E119', 'F019', 'N390', 'C900', 'X70', 'V892', 'I255', 'X95']


In [ ]:
df45 = df3.head(45)
df45 = pd.DataFrame(df45)
code_list_45 = df45["icd_death_code"].tolist()
print(code_list_45)

['I251', 'C349', 'J449', 'I219', 'G309', 'F03', 'I500', 'I64', 'I250', 'J189', 'C509', 'C259', 'C189', 'A419', 'E149', 'C61', 'I119', 'G20', 'C80', 'I48', 'J440', 'I10', 'X44', 'X42', 'N185', 'J690', 'K746', 'I429', 'C679', 'K703', 'W19', 'C159', 'C719', 'I350', 'E142', 'I469', 'C64', 'J841', 'C56', 'I619', 'I110', 'X74', 'E119', 'F019', 'N390']


In [ ]:
df40 = df3.head(40)
df40 = pd.DataFrame(df40)
code_list_40 = df40["icd_death_code"].tolist()
print(code_list_40)

['I251', 'C349', 'J449', 'I219', 'G309', 'F03', 'I500', 'I64', 'I250', 'J189', 'C509', 'C259', 'C189', 'A419', 'E149', 'C61', 'I119', 'G20', 'C80', 'I48', 'J440', 'I10', 'X44', 'X42', 'N185', 'J690', 'K746', 'I429', 'C679', 'K703', 'W19', 'C159', 'C719', 'I350', 'E142', 'I469', 'C64', 'J841', 'C56', 'I619']


In [ ]:
df35 = df3.head(35)
df35 = pd.DataFrame(df35)
code_list_35 = df35["icd_death_code"].tolist()
print(code_list_35)

['I251', 'C349', 'J449', 'I219', 'G309', 'F03', 'I500', 'I64', 'I250', 'J189', 'C509', 'C259', 'C189', 'A419', 'E149', 'C61', 'I119', 'G20', 'C80', 'I48', 'J440', 'I10', 'X44', 'X42', 'N185', 'J690', 'K746', 'I429', 'C679', 'K703', 'W19', 'C159', 'C719', 'I350', 'E142']


In [ ]:
df30 = df3.head(30)
df30 = pd.DataFrame(df30)
code_list_30 = df30["icd_death_code"].tolist()
print(code_list_30)

['I251', 'C349', 'J449', 'I219', 'G309', 'F03', 'I500', 'I64', 'I250', 'J189', 'C509', 'C259', 'C189', 'A419', 'E149', 'C61', 'I119', 'G20', 'C80', 'I48', 'J440', 'I10', 'X44', 'X42', 'N185', 'J690', 'K746', 'I429', 'C679', 'K703']


In [ ]:
df25 = df3.head(25)
df25 = pd.DataFrame(df25)
code_list_25 = df25["icd_death_code"].tolist()
print(code_list_25)

['I251', 'C349', 'J449', 'I219', 'G309', 'F03', 'I500', 'I64', 'I250', 'J189', 'C509', 'C259', 'C189', 'A419', 'E149', 'C61', 'I119', 'G20', 'C80', 'I48', 'J440', 'I10', 'X44', 'X42', 'N185']


In [ ]:
df20 = df3.head(20)
df20 = pd.DataFrame(df20)
code_list_20 = df20["icd_death_code"].tolist()
print(code_list_20)

['I251', 'C349', 'J449', 'I219', 'G309', 'F03', 'I500', 'I64', 'I250', 'J189', 'C509', 'C259', 'C189', 'A419', 'E149', 'C61', 'I119', 'G20', 'C80', 'I48']


In [ ]:
df15 = df3.head(15)
df15 = pd.DataFrame(df15)
code_list_15 = df15["icd_death_code"].tolist()
print(code_list_15)

['I251', 'C349', 'J449', 'I219', 'G309', 'F03', 'I500', 'I64', 'I250', 'J189', 'C509', 'C259', 'C189', 'A419', 'E149']


In [ ]:
df10 = df3.head(10)
df10 = pd.DataFrame(df10)
code_list_10 = df10["icd_death_code"].tolist()
print(code_list_10)

['I251', 'C349', 'J449', 'I219', 'G309', 'F03', 'I500', 'I64', 'I250', 'J189']


In [ ]:
df5 = df3.head(5)
df5 = pd.DataFrame(df5)
code_list_5 = df5["icd_death_code"].tolist()
print(code_list_5)

['I251', 'C349', 'J449', 'I219', 'G309']


In [ ]:
death_2015_final_5 = death_2015_final.filter(death_2015_final.icd_death_code.isin(code_list_5))
death_2015_final_5.show()

+--------------+--------+---------+--------------+---+---+--------------+--------------------+-----------------+--------------+---------------+----+--------+------------------+------------------+------------------+
|icd_death_code|resident|education|month_of_death|sex|age|marital_status|day_of_week_of_death|current_data_year|injury_at_work|manner_of_death|race|hispanic|entity_condition_1|entity_condition_2|entity_condition_3|
+--------------+--------+---------+--------------+---+---+--------------+--------------------+-----------------+--------------+---------------+----+--------+------------------+------------------+------------------+
|          C349|       1|        6|             1|  1| 70|             2|                   2|             2015|             3|              7|   1|       6|            11I469|            21R042|            31C349|
|          G309|       1|        3|             1|  2| 91|             3|                   6|             2015|             3|             

In [ ]:
#this total number of rows matches the total for the top 5 causes of death, thi saccounts for 24.2% of the original data frame
death_2015_final_5.count()

651573

In [ ]:
# 36.9%
death_2015_final_10 = death_2015_final.filter(death_2015_final.icd_death_code.isin(code_list_10))
death_2015_final_10.count()

995554

In [ ]:
# 44.1%
death_2015_final_15 = death_2015_final.filter(death_2015_final.icd_death_code.isin(code_list_15))
death_2015_final_15.count()

1187783

In [ ]:
# 49.1%
death_2015_final_20 = death_2015_final.filter(death_2015_final.icd_death_code.isin(code_list_20))
death_2015_final_20.count()

1323877

In [ ]:
# 52.9%
death_2015_final_25 = death_2015_final.filter(death_2015_final.icd_death_code.isin(code_list_25))
death_2015_final_25.count()

1426272

In [ ]:
# 56.2%
death_2015_final_30 = death_2015_final.filter(death_2015_final.icd_death_code.isin(code_list_30))
death_2015_final_30.count()

1514013

In [ ]:

death_2015_final_35 = death_2015_final.filter(death_2015_final.icd_death_code.isin(code_list_35))
death_2015_final_35.count()

1588750

In [ ]:
#58.96%
death_2015_final_40 = death_2015_final.filter(death_2015_final.icd_death_code.isin(code_list_40))
death_2015_final_40.count()

1659355

In [ ]:
# 63.97%
death_2015_final_45 = death_2015_final.filter(death_2015_final.icd_death_code.isin(code_list_45))
death_2015_final_45.count()

1723541

In [ ]:
# 66.2%
death_2015_final_50 = death_2015_final.filter(death_2015_final.icd_death_code.isin(code_list_50))
death_2015_final_50.count()

1782756

In [ ]:
# 68.2%
death_2015_final_55 = death_2015_final.filter(death_2015_final.icd_death_code.isin(code_list_55))
death_2015_final_55.count()

1838247

In [ ]:
# 70.1%
death_2015_final_60 = death_2015_final.filter(death_2015_final.icd_death_code.isin(code_list_60))
death_2015_final_60.count()

1889442

In [ ]:
death_2015_final_60 = death_2015_final.filter(death_2015_final.icd_death_code.isin(code_list_60))
death_2015_final_60.show()

+--------------+--------+---------+--------------+---+---+--------------+--------------------+-----------------+--------------+---------------+----+--------+------------------+------------------+------------------+
|icd_death_code|resident|education|month_of_death|sex|age|marital_status|day_of_week_of_death|current_data_year|injury_at_work|manner_of_death|race|hispanic|entity_condition_1|entity_condition_2|entity_condition_3|
+--------------+--------+---------+--------------+---+---+--------------+--------------------+-----------------+--------------+---------------+----+--------+------------------+------------------+------------------+
|          I500|       1|        3|             1|  1| 84|             2|                   1|             2015|             3|              7|   1|       6|            11I500|            61L031|                 0|
|          C349|       1|        6|             1|  1| 70|             2|                   2|             2015|             3|             

In [ ]:
death_2015_final_60 = death_2015_final_60.na.replace(["E119", "E142", "F019", "I509", "I694","J440", "X42"], ["E149","E149", "F03", "I500", "I64", "J449", "X44"], "icd_death_code")
death_2015_final_60.show()

+--------------+--------+---------+--------------+---+---+--------------+--------------------+-----------------+--------------+---------------+----+--------+------------------+------------------+------------------+
|icd_death_code|resident|education|month_of_death|sex|age|marital_status|day_of_week_of_death|current_data_year|injury_at_work|manner_of_death|race|hispanic|entity_condition_1|entity_condition_2|entity_condition_3|
+--------------+--------+---------+--------------+---+---+--------------+--------------------+-----------------+--------------+---------------+----+--------+------------------+------------------+------------------+
|          I500|       1|        3|             1|  1| 84|             2|                   1|             2015|             3|              7|   1|       6|            11I500|            61L031|                 0|
|          C349|       1|        6|             1|  1| 70|             2|                   2|             2015|             3|             

In [ ]:
death_2015_final_60.show()

+--------------+--------+---------+--------------+---+---+--------------+--------------------+-----------------+--------------+---------------+----+--------+------------------+------------------+------------------+
|icd_death_code|resident|education|month_of_death|sex|age|marital_status|day_of_week_of_death|current_data_year|injury_at_work|manner_of_death|race|hispanic|entity_condition_1|entity_condition_2|entity_condition_3|
+--------------+--------+---------+--------------+---+---+--------------+--------------------+-----------------+--------------+---------------+----+--------+------------------+------------------+------------------+
|          I500|       1|        3|             1|  1| 84|             2|                   1|             2015|             3|              7|   1|       6|            11I500|            61L031|                 0|
|          C349|       1|        6|             1|  1| 70|             2|                   2|             2015|             3|             

In [ ]:
death_2014_final_60 = death_2014_final.filter(death_2014_final.icd_death_code.isin(code_list_60))
death_2014_final_60.show()

+--------------+--------+---------+--------------+---+---+--------------+--------------------+-----------------+--------------+---------------+----+--------+------------------+------------------+------------------+
|icd_death_code|resident|education|month_of_death|sex|age|marital_status|day_of_week_of_death|current_data_year|injury_at_work|manner_of_death|race|hispanic|entity_condition_1|entity_condition_2|entity_condition_3|
+--------------+--------+---------+--------------+---+---+--------------+--------------------+-----------------+--------------+---------------+----+--------+------------------+------------------+------------------+
|           I64|       1|        2|             1|  1| 87|             2|                   4|             2014|             3|              7|   1|       6|             11I64|                 0|                 0|
|          I250|       1|        2|             1|  1| 58|             4|                   3|             2014|             3|             

In [ ]:
death_2014_final_60 = death_2014_final_60.na.replace(["E119", "E142", "F019", "I509", "I694","J440", "X42"], ["E149","E149", "F03", "I500", "I64", "J449", "X44"], "icd_death_code")
death_2014_final_60.show()

+--------------+--------+---------+--------------+---+---+--------------+--------------------+-----------------+--------------+---------------+----+--------+------------------+------------------+------------------+
|icd_death_code|resident|education|month_of_death|sex|age|marital_status|day_of_week_of_death|current_data_year|injury_at_work|manner_of_death|race|hispanic|entity_condition_1|entity_condition_2|entity_condition_3|
+--------------+--------+---------+--------------+---+---+--------------+--------------------+-----------------+--------------+---------------+----+--------+------------------+------------------+------------------+
|           I64|       1|        2|             1|  1| 87|             2|                   4|             2014|             3|              7|   1|       6|             11I64|                 0|                 0|
|          I250|       1|        2|             1|  1| 58|             4|                   3|             2014|             3|             

In [ ]:
df = death_2015_final_60.unionByName(death_2014_final_60)
df.show()

+--------------+--------+---------+--------------+---+---+--------------+--------------------+-----------------+--------------+---------------+----+--------+------------------+------------------+------------------+
|icd_death_code|resident|education|month_of_death|sex|age|marital_status|day_of_week_of_death|current_data_year|injury_at_work|manner_of_death|race|hispanic|entity_condition_1|entity_condition_2|entity_condition_3|
+--------------+--------+---------+--------------+---+---+--------------+--------------------+-----------------+--------------+---------------+----+--------+------------------+------------------+------------------+
|          I500|       1|        3|             1|  1| 84|             2|                   1|             2015|             3|              7|   1|       6|            11I500|            61L031|                 0|
|          C349|       1|        6|             1|  1| 70|             2|                   2|             2015|             3|             

In [ ]:
df.count()

3731481

In [ ]:
death_2015_final_60.count()

1889442

In [ ]:
death_2014_final_60.count()

1842039